In [438]:
# Import general libraries
import geopandas as gpd
import pandas as pd

# Import Bokeh libraries
from bokeh.tile_providers import get_provider, Vendors
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure,output_file, save


#### Read files and set coordinate reference system

In [439]:
# Read files
stations_population = gpd.read_file("population_data/stations_population.shp")
stations = gpd.read_file("stations_data/stations.shp")
rails = gpd.read_file("railway_data/railways.shp")

# Set crs to EPSG:3857
stations_population = stations_population.to_crs(epsg=3857)
stations = stations.to_crs(epsg=3857)
rails = rails.to_crs(epsg=3857)

# Show first rows of stations_population
stations_population.head(2)

,Name,Opened,Type,pop1997,pop1998,pop1999,pop2000,pop2001,pop2002,pop2003,...,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,geometry
0,Espoo,1903,Train,3604.0,3642.0,2195.0,2142.0,3484.0,3469.0,3467.0,...,3637.0,4003.0,3952.0,4229.0,4256.0,4356.0,4206.0,4400.0,4173.0,"POLYGON ((2745784.758 8445625.162, 2745779.943..."
1,Helsinki Center Station,1862,Train/Metro,546.0,518.0,711.0,713.0,497.0,483.0,474.0,...,479.0,523.0,533.0,562.0,556.0,550.0,545.0,581.0,594.0,"POLYGON ((2777450.701 8437894.974, 2777445.885..."


#### Convert stations population dataframe so that years are in own column and remove pop+year columns

In [455]:
def years_to_columns(df, year):
    # Select columns
    station_population = df[["Name", "Opened", "Type", "pop"+str(year), "geometry"]]
    # Create column Year and set value
    station_population["DataYear"] = year
    # Rename column population+year as Population
    station_population = station_population.rename(columns={"pop"+str(year): "Population"})
    # Change population type to integer
    station_population = station_population.astype({"Population": int})
    # Return value
    return station_population


# Get columns names
columns = stations_population.columns.tolist()

# Filter columns list and leave only population columns (they have year eg. pop1997)
pop_years = list(filter(lambda x: 'pop' in x, columns))

# Remove pop and leave only years
tmp_list=[]
for name in pop_years:
    year = name[3:7]
    year = int(year)
    tmp_list.append(year)

# Save years to pop_years
pop_years = tmp_list

# Drop year 1997 from to list and use it to initialize population geodataframe
pop_years.remove(1997)
population = years_to_columns(stations_population, 1997)

# Loop through pop years, create annual dataframes and join dataframe together
for year in pop_years:
    # Create dataframe population_tmp, which has stations population from one year
    population_tmp = years_to_columns(stations_population, year)
    # Concat new dataframe to dataframe population
    population = pd.concat([population, population_tmp])

# Save dataframe population as stations_population
stations_population = population

# Show first rows of dataframe population
stapopulation

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Name,Opened,Type,Population,geometry,DataYear
0,Espoo,1903,Train,3604,"POLYGON ((2745784.758 8445625.162, 2745779.943...",1997
1,Helsinki Center Station,1862,Train/Metro,546,"POLYGON ((2777450.701 8437894.974, 2777445.885...",1997
2,Hiekkaharju,1931,Train,2513,"POLYGON ((2789487.121 8467534.963, 2789482.305...",1997
3,Huopalahti,1910,Train,3730,"POLYGON ((2772325.997 8448441.335, 2772321.181...",1997
4,Ilmala,1967,Train,1006,"POLYGON ((2775229.432 8446094.524, 2775224.616...",1997
...,...,...,...,...,...,...
56,Siilitie,1982,Metro,2176,"POLYGON ((2788919.503 8445637.484, 2788914.687...",2019
57,Sörnäinen,1984,Metro,15151,"POLYGON ((2779633.119 8441676.764, 2779628.304...",2019
58,Tapiola,2017,Metro,3200,"POLYGON ((2762330.619 8438838.784, 2762325.804...",2019
59,Urheilupuisto,2017,Metro,2082,"POLYGON ((2759488.188 8438689.047, 2759483.372...",2019


#### Convert rails GeoDataFrame to ColumnDataSource

In [441]:
# Create function getlinecoords
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

# Get x and y coordinates
rails['x'] = rails.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
rails['y'] = rails.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

# Drop all the columns except osmid coordinates x and y
rails = rails[["osmid", "x", "y"]]

# Convert to ColumnDataSource
rails = ColumnDataSource(rails)

#### Convert stations GeoDataFrame to ColumnDataFrame

In [442]:
# Create function getpointcoords
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

# Get x and y coordinates
stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)

# Calculate y coordinates
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

# Drop all the columns except osmid coordinates x and y
#stations = stations.drop('geometry', axis=1)
stations = stations[["x", "y"]]

# Convert to ColumnDataSource
stations = ColumnDataSource(stations)

#### Convert stations_population from GeoDataFrame to geojson

In [443]:
# Convert grid geodataframe to geoJSON for plotting
geosource = GeoJSONDataSource(geojson = stations_population.to_json())

#### Convert stations_population from GeoDataFrame to geojson

In [444]:
geosource_population = GeoJSONDataSource(geojson = stations_population.to_json())

#### Create Bokeh visualization

In [445]:
# Set outputfile name
output_file("bokeh_map.html")

# Define fill colors palette
palette = brewer['BuGn'][6]

# Reverse fill colors palette order so higher values have darker colors
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 12000)

# Create empty figure object
p = figure(title = 'Population living in 500 meters range from metro and train stations', 
           plot_height = 700 ,
           plot_width = 900, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Get background map provider
tile_provider = get_provider('CARTODBPOSITRON')

# Add background map
p.add_tile(tile_provider)

# Add the rails to the map
p.multi_line('x', 'y', source=rails, color='gray', line_width=1)

# Remove axis
p.axis.visible = False





# ------ TOIMII -------

# Add patch renderer to figure.
grids = p.patches('xs','ys', source = geosource, fill_color = {'field' :'pop2019', 'transform' : color_mapper},
                   line_color = "black", line_width = 0.25, fill_alpha = 0.8)











# ------ TOIMII -------


# Add the stations to the map as black circle over station
p.circle('x', 'y', source=stations, color='black', size=3)

# Define year slider
year_slider = Slider(start=1997, end=2019, value=1997, step=1, title="Year")

# Define custom labels for color bar
tick_labels = {'0': '0', '2000': '2,000',
               '4000':'4,000', '6000':'6,000',
               '8000':'8,000', '10000':'10,000','12000':'12,000+'}

# Create color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 10,
                     width = 15, height = 600,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = "vertical", major_label_overrides = tick_labels)

# Create hover tool
p.add_tools(HoverTool(renderers = [grids], 
                      tooltips = [('Station','@Name'),('Type','@Type'),
                                  ('Opened','@Opened'), ('Population','@pop2019')]))

# Add color bar
p.add_layout(color_bar, "right")

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p, widgetbox(year_slider))

# Show map
show(layout)